In [ ]:
%load_ext autoreload
%autoreload 2

# Exercise 4

<img src='./images/04.png' width=800>

In [ ]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns03_4'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise_4')

2025/04/09 08:42:38 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/674194671375701503', creation_time=1744188158264, experiment_id='674194671375701503', last_update_time=1744188158264, lifecycle_stage='active', name='Exercise_1', tags={}>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.optim as optim
from torchinfo import summary
from utils import train_network, accuracy_score_wrapper


In [ ]:
# C = cifar_train[0][0].shape[0]
C = 3
filter = 16
K = 3
# w = cifar_train[0][0].shape[1]
# h = cifar_train[0][0].shape[2]
w , h = 32, 32
# classes = cifar_train[0][1].shape[0]
classes = 10
def build_model(num_conv_layers,
                num_pool_layers,
                num_hidden_layer=2, 
                init_hidden_size=512, 
                decay_factor=2,
                activation=nn.ReLU()):
    layers =[]
    in_channels = C
    out_channels = 32
    if num_pool_layers:
        pool_interval = max(1, num_conv_layers // (num_pool_layers + 1))
    else:
        pool_interval = num_conv_layers + 1
    
    currnet_pool_rounds = 0
    for i in range(num_conv_layers):
        layers.append(nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels,
            kernel_size=3,
            padding=3//2))
        layers.append(activation)
        in_channels = out_channels
        if (i+1) % pool_interval == 0 and currnet_pool_rounds < num_pool_layers:
            layers.append(nn.MaxPool2d(kernel_size=2))
            currnet_pool_rounds += 1
            out_channels *= 2
    final_w = w // (2 ** num_pool_layers)
    final_h = h // (2 ** num_pool_layers)
    fc_layers = []
    # Compute the number of features after flattening.
    in_features = in_channels * final_w * final_h

    fc_layers.append(nn.Flatten())

    if num_hidden_layer == 0:
        # Directly classify without extra hidden layers.
        fc_layers.append(nn.Linear(in_features, classes))
    else:
        # First FC layer: from flattened output to initial hidden size.
        fc_layers.append(nn.Linear(in_features, init_hidden_size))
        fc_layers.append(nn.ReLU(inplace=True))
        
        # Set the current hidden size that will be reduced in subsequent layers.
        current_hidden_size = init_hidden_size

        # Add additional hidden layers with decreasing size.
        for layer in range(1, num_hidden_layer):
            # Compute new hidden size with decay.
            new_hidden_size = max(10, current_hidden_size // decay_factor)
            fc_layers.append(nn.Linear(current_hidden_size, new_hidden_size))
            fc_layers.append(nn.ReLU(inplace=True))
            current_hidden_size = new_hidden_size

        # Final classification layer from the last hidden dimension to the number of classes.
        fc_layers.append(nn.Linear(current_hidden_size, classes))

    classifier = nn.Sequential(*fc_layers)
    model = nn.Sequential(*layers, classifier)
    return model

In [ ]:
batch_size = 256

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
cifar_train = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transform,
    train=True,)
cifar_test = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transform,
    train=False,)

cifar_train_loader = DataLoader(
    cifar_train,
    shuffle=True,
    batch_size=batch_size,
    )

cifar_test_loader = DataLoader(
    cifar_test,
    batch_size=batch_size,
    )

100%|██████████| 170M/170M [00:16<00:00, 10.2MB/s] 


Extracting ./data_cifar/cifar-10-python.tar.gz to ./data_cifar
Files already downloaded and verified


In [ ]:
transforms_augmentation = {
    'RandomAffine1': transforms.RandomAffine(degrees=10),
    'RandomAffine2': transforms.RandomAffine(degrees=0, translate=(0.1,0.1)),
    'RandomAffine3': transforms.RandomAffine(degrees=0, shear=15),
    'RandomHorizontalFlip': transforms.RandomHorizontalFlip(p=0.5),
    'RandomVerticalFlip': transforms.RandomVerticalFlip(p=0.2),
    'RandomPerspective': transforms.RandomPerspective(p=.2),   
    'ColorJitter': transforms.ColorJitter(brightness=0.5, contrast=0.5),
    }

In [ ]:
cifar_augmented_train_loader = {}
for trfm_name, trfm in transforms_augmentation.items():
    train_transforms = transforms.Compose(
    [trfm,
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]    
    )
    cifar_augmented_train = torchvision.datasets.CIFAR10(
    f'./data_augmented_cifar_{trfm_name}',
    download=True,
    transform=train_transforms,
    train=True,)

    cifar_augmented_train_loader[trfm_name] = DataLoader(
        cifar_augmented_train,
        shuffle=True,
        batch_size=batch_size,)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 10
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}

In [ ]:

# model = model_cnn_pool(D=32*32, C=3, filters=128, K=3, classes=10)
train_loaders = {'without_aug': cifar_train_loader,
                 **cifar_augmented_train_loader}

In [ ]:
for aug_name, train_loader in train_loaders.items():
    print(aug_name)
    model = build_model(num_conv_layers=4,
            num_pool_layers=2,
            num_hidden_layer=2, 
            init_hidden_size=512, 
            decay_factor=2,
            activation=nn.ReLU())
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    params['optimizer'] = optimizer.defaults
    
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model)))
    with mlflow.start_run(nested=True, run_name=aug_name):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=cifar_test_loader,
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

<img src="./images/valid_acc_aug.png">

<img src="./images/valid_loss aug.png">